In [1]:
# import packages
from numpy.core.numeric import NaN
from numpy.lib.arraypad import _set_reflect_both
from numpy.lib.utils import byte_bounds
from pandas.core.frame import DataFrame
import requests
import datetime as dt
import pandas as pd
import json
from requests.api import request
from geopy.geocoders import Nominatim
import numpy as np
import math
import bisect
from scipy.stats import zscore
from numpy import add
from d01_data.get_weather_data import getload_weather, get_solar_irradiance
from d01_data.get_address_data import get_reference_data as getref
import d01_data.get_topo_data as topo_raw
import d02_intermediate.create_topo_int as topo_int
import d03_processing.topo_process as topo_pro
from d02_intermediate.est_albedo import albedo
import os
from os.path import dirname, abspath
import sys 
import d00_utils.id_management as idmanage
from ast import literal_eval as make_tuple
import pickle
import d02_intermediate.geometry3d as sh
import d07_visualisation.create_geometries as creategeo
import geopandas as gpd
from itertools import product
from geojson import Feature, Point, Polygon, MultiPolygon, FeatureCollection
import matplotlib.pyplot as plt
import shapely.geometry as sg
from d02_intermediate.geometry3d import inc_angle
from d02_intermediate.est_incidence_on_panel import radiation_incidence_on_panel as incidence_on_panel
# ignore pd warnings
pd.options.mode.chained_assignment = None 

In [2]:
# set parameters for energy consumption
# might be used to calculate optimal locations
use_standard_consumption = True
# use standard values for germany if needed
if use_standard_consumption:
    # set average energy consumption for june and january in kWh(source: https://musterhaushalt.de)
    power_usage_january_total = 299
    power_usage_july_total = 249
    # set energy consumption values by times of day in Wh
    # consumption night from 24:00 to 06:00
    consumption_night = 14*70
    # morning = 06:00 to 12:00
    consumption_morning = 36*70
    # afternoon = 12:00 to 18:00
    consumption_afternoon = 36*70
    # evening = 18:00 to 24:00
    consumption_evening = 43*70
    # set the month of the measuring date for the times of day
    measuring_month = 'september'
# find the fitting category for the consumption profile: consumption_type
consumption_type = 'max total'


In [3]:
# set some panel parameters
#  set if the panel mounts can adjust the tilt and alignment
adjustable_mounts = True
# set panel sizes in mm
panel_length = 1660
panel_width = 990
# set the adjustable range in mm: mount_leeway
mount_leeway = 400
if adjustable_mounts:
    max_tilt_adj = math.degrees(math.asin(mount_leeway/panel_length))
    max_align_adj = math.degrees(math.asin(mount_leeway/panel_width))

# temperature coefficient P_mpp or gamma
temp_coeff = -.43
# compute losses by technical issues (Inverter, mismatch, LID, age (15y))
loss_technical = 1-(.97*.98*.985*.9625)
# set panel efficiency: panel_efficiency (under STC)
panel_efficiency = .17

In [4]:
# set id for round (string)
process_id = 'test'
# choose if locally saved data will be used (if available)
use_available = True
# set degree delta for resolution of topo requests, don't change this value, other values are not possible currently
airmaps_degree_delta = float(0.000277778)
# choose the locations
location = 'thüringen'
# choose the resolution
resolution = 24
# create an updated process_id
process_id = process_id+'_'+location+'_res_'+str(resolution)

In [5]:
# create a reference dict for the location
# info from the dict will be used for the request of weather data e.g.
reference_dict = getref(location)

In [6]:
# update the location dictionary
# set if the update will be forced
force_update_dict = False
# set if a new dict has to be created (the old one gets deleted -_- ) 
create_new_dict = False
# set the load/save path
path = '../references/location_dictionary.pkl'
# load the current dict if available
if os.path.exists(path) and not create_new_dict:
    with open(path,'rb') as f:
        loc_dict=pickle.load(f)
else:
    loc_dict = {}

# check if the location is already included in the dict
if (not location in loc_dict) or force_update_dict:
    # if the location is new, add the reference data
    loc_dict[location] = getref(location)
    # save it
    with open(path,'wb') as f:
        pickle.dump(loc_dict,f)

In [7]:
# import weather data
weather_df = getload_weather(process_id,location=location,use_available=True, resolution=resolution)
# drop useless columns
weather_dropped_df = weather_df.drop(['lat','lon','id'],axis=1)

available weather data will be loaded. change use_available or process_id to request new data


In [8]:
# code in this cell is outdated, kept for documentation purposes

# # drop rows with zero radiation
# # find all rows that correspond to a time of zero radiation
# is_rad = weather_df['parameter']=='global_rad:W'
# is_zero = weather_df['value']==0
# # save the data to drop
# to_drop = weather_df[is_zero & is_rad]
# # drop the rows
# weather_dropped_df = weather_df.drop(to_drop.index)
# # drop unnecessary coordinates from weather_dropped_df
# weather_dropped_df.drop(['lat','lon','id'],inplace=True,axis=1)

In [9]:
# prepare dataframe to save all ids and block statuses
ids_all_df = idmanage.id_manage_df(weather_dropped_df,id_col='id_tuple')
ids_allowed = idmanage.id_allow_df(ids_all_df)
# empty block dataframe
block_df = pd.DataFrame(columns=['id_tuple','blocked','block reason'])

In [10]:
# block locations outside of the chosen area

# access reference polygon: reference
reference = loc_dict[location]['polygon']

# create a dataframe with currently allowed ids
loc_gdf = pd.DataFrame(ids_allowed)
# add a Point column with shapely Points
loc_gdf['geometry'] = loc_gdf.apply(lambda x: sg.Point((x['id_tuple'][1],x['id_tuple'][0])), axis=1)
# update the block_df
block_df = idmanage.update_id_block_df(loc_gdf,block_df,'outside of',reference,'outside of '+location, block_col='geometry')
# update the other id management structures
ids_all_df = idmanage.updateblocks_idmanage_df(block_df, ids_all_df)
ids_allowed = idmanage.id_allow_df(ids_all_df)
# drop weather data from blocked locations
weather_dropped_df = weather_dropped_df[weather_dropped_df['id_tuple'].isin(ids_allowed)]

In [11]:
# reformat the dataframe so it has a single row per time and location with all of the weather data
# get a list of the unique parameters
parameters = weather_dropped_df.parameter.unique()
# create a sub_df for for each parameter
# collect subs in a list: sub_df_list
sub_df_list =[]
for param in parameters:
    # get a sub_df
    sub_df = weather_dropped_df[weather_dropped_df['parameter']==param]
    # reset the index, then access the parameter name in the sub
    sub_df.reset_index(inplace=True,drop=True)
    param = sub_df.parameter[0]
    # rename the value column with a more describing name, drop the parameter column
    sub_df.columns = ['date',param,param+' value','id_tuple']
    sub_df = sub_df[['date', 'id_tuple',param+' value']]
    # append the sub to the sub list
    sub_df_list.append(sub_df)
# merge all sub_dfs to create the reformated df
weather_reformatted_df = sub_df_list[0]
for sub in sub_df_list[1:]:
    weather_reformatted_df = pd.merge(weather_reformatted_df, sub, on=['id_tuple','date'],suffixes=('',''))


In [12]:
# simplify the col names of weather_reformatted_df
# take the col names, create empty list for new names
cols = weather_reformatted_df.columns
new_cols = []
# iterate over the vol names
for name in cols:
    pos = name.find(':')
    # check if there are units given, if yes drop them
    if pos >0:
        name = name[:pos]
    # collect new col names
    new_cols.append(name)
# rename the cols of weather_reformatted_df
weather_reformatted_df.columns=new_cols


In [13]:
# get for the tilt & align values
from d00_utils.tiltalign_adjust import tilt_options,align_options
from collections import deque
tilt_opts = tilt_options(power_usage_july_total,power_usage_january_total)
align_suggs = align_options(consumption_night,consumption_morning,consumption_afternoon,consumption_evening)


# loop over allowed ids and tilt, align options and find out, how much daily output after high variance factors is available
highvar_allids=[]

for id in ids_allowed:
    test = weather_reformatted_df[weather_reformatted_df.id_tuple== id]
    # reset the index for better integration
    test.reset_index(drop=True,inplace=True)
    # gather and process low variance data for the chosen id
    # begin with extracting latitude and longitude
    lat = id[0]
    lon = id[1]

    # you will need elevation data in the surrounding area
    from d01_data.get_topo_data import elevation_carpet
    carpet = elevation_carpet(lat,lon)
    # extract the necessary info from the carpet
    from d02_intermediate.create_topo_int import carpet2df, ground_grad_ns,ground_grad_ew,carpet_characterics
    elevation_df = carpet2df(carpet)
    characteristics = carpet_characterics(carpet)
    ns_grad = ground_grad_ns(elevation_df,characteristics['ns_stepsize'])
    ew_grad = ground_grad_ns(elevation_df,characteristics['ew_stepsize'])
    
    # get the tilt suggestions
    natural_tilt = math.degrees(math.atan(ns_grad))
    tilt_suggs = [natural_tilt] + tilt_opts
    
    # save the datetime info in a more fitting column
    test['datetime'] = test['date']
    # drop the time info drom the date column
    test['date'] = test.apply(lambda x: x.date.date(),axis=1)

    # add some geometric values
    from d02_intermediate.geometry3d import sun_geo as sg
    # get info about sunheight and azimut for each row
    test['sungeo'] = test.apply(lambda x: sg(lat,lon,x.datetime),axis=1)
    test['azimut'] = test.apply(lambda x: x.sungeo.get('azimut'),axis=1)
    test['sunheight'] = test.apply(lambda x: x.sungeo.get('sunheight refracted'),axis=1)
    test.drop('sungeo',inplace=True,axis=1)
    # get data about the albedo
    test.loc[:,'albedo'] = test.apply(lambda x: albedo(lat,lon),axis=1)

    # ADD IN ITERATION FOR TILTALIGN OPTIONS HERE

    for alignment in align_suggs:
        for tilt in tilt_suggs:

            # time for high variance factors which vary for each datetime
            # all this data will be put into additional rows

            # compute the gradients of the panel
            ns_grad = math.tan(math.radians(tilt))
            ew_grad = math.tan(math.radians(alignment))

            # get data about the angle of incidence
            test['angle'] = test.apply(lambda x: inc_angle(x.azimut,x.sunheight,ns_grad,ew_grad),axis=1)

            # iterate over the unique dates, get irradiance_toa info for each date and save in a dict
            toa_dict = {}
            for date in test.date.unique():
                toa_dict[date]= get_solar_irradiance(date)
            # add the total solar irradiance to the dataframe
            test['solar_irradiance'] = test.apply(lambda x: toa_dict[x.date],axis=1)

            # compute the incidence on the panel
            test['incidence'] = test.apply(lambda x: incidence_on_panel(x.global_rad,x.direct_rad,x.diffuse_rad,x.solar_irradiance,x.sunheight,tilt,x.albedo,x.angle),axis=1)


            # get the high variance losses
            # import the needed packages
            from d02_intermediate.weather_int import est_snowdepth_panel, est_paneltemp, loss_by_snow, loss_by_temp, est_soiling_loss_data
            # begin with the snow
            # estimate the snow depth on the panel
            test['snow_depth_panel'] = test.apply(lambda x: est_snowdepth_panel(x['snow_depth'],tilt),axis=1)
            # compute the generated losses
            test['loss_snow'] = test.apply(lambda x: loss_by_snow(x.snow_depth_panel),axis=1)

            # loss by panel temp
            # estimate the panel temp
            test['panel_temp'] = test.apply(lambda x: est_paneltemp(x.t_2m,x.wind_speed_10m),axis=1)
            # estimate the resulting loss in output
            test['loss_temp']  = test.apply(lambda x: loss_by_temp(x.panel_temp,temp_coeff),axis=1)

            # loss by soiling
            # create a soil_df with soiling data
            soil_df = est_soiling_loss_data(natural_tilt, test,'total_aod_550nm','precip_1h','datetime',24)
            # add the soil loss data to the full dataframe
            test[['hrs_since_rain','aod_mean','loss_soil']] = soil_df[['hrs_since_rain','aod_mean','soil loss pct']]
            # all high variance factors are included

            # compute the total loss by those factors and the output after those factors
            test['loss_highvar'] = test.apply(lambda x: 1-(1-x.loss_snow)*(1-x.loss_temp)*(1-x.loss_soil),axis=1)
            test['output_highvar'] = test.apply(lambda x: x.incidence*(1-x.loss_highvar),axis=1)


            # get the total output of the location per day
            # compute the covered timespan
            tdelta = test.datetime.iloc[-1]-test.datetime.iloc[0]
            timespan = tdelta.days+tdelta.seconds/(24*3600)
            # get the total output after high variance loss factors
            output_highvar_total = test.output_highvar.sum()
            # compute the mean output per day
            output_highvar_daily = output_highvar_total/timespan

            # collect the most relevant data in a dict
            relevant = ['loss_temp','loss_snow','loss_soil','loss_highvar']
            highvar_summary_dict = {}
            for item in relevant:
                highvar_summary_dict[item] = test[item].mean()

            # collect the data
            # relevant: loss factors, loss total, output daily +  tilt, align, id
            highvar_list = [id,
                            tilt,
                            alignment,
                            output_highvar_daily,
                            test.loss_highvar.mean(),
                            test.loss_temp.mean(),
                            test.loss_snow.mean(),
                            test.loss_soil.mean(),
                            test.angle.min()]
            highvar_allids.append(highvar_list)

# create a new dataframe from all the data
highvar_allids_df = pd.DataFrame(highvar_allids)
# name the cols
highvar_allids_df.columns=['id_tuple','tilt','alignment','output_highvar_daily','loss_highvar','loss_temp','loss_snow','loss_soil','angle']

97.634026610204 77.46623499996447
98.8349747253786 77.46623499996447
98.99410498500431 77.46623499996447
107.5936311307802 77.46623499996447
108.31345621182825 77.46623499996447
108.21887114084582 77.46623499996447
117.54847594992621 77.46623499996447
117.95034522097986 77.46623499996447
117.68986597934027 77.46623499996447


In [ ]:
id_loc =2
subset = highvar_allids_df[highvar_allids_df['id_tuple'] == ids_allowed.iloc[id_loc]]
max_daily = subset.output_highvar_daily.max()
subset[subset.output_highvar_daily>.9*max_daily]

In [ ]:
test[['datetime','id_tuple','loss_snow','loss_soil','loss_temp','loss_highvar','incidence','output_highvar']]

In [ ]:
# turn weather dropped into a gdf with point geometries
gdf = weather_dropped_df
gdf['geometry'] = gdf.apply(lambda x: Point(x['id_tuple']),axis=1)
crs = 'epsg:4326'
gdf = gpd.GeoDataFrame(gdf,crs=crs,geometry=gdf.geometry)

In [ ]:
# this cell contains outdated code for documentation purposes

# force_polyrequest = True
# # setting if a new grid will be requested
# force_new_grid = False
# # check the number of unique locations
# # if the number of unique locations is too high for a relatively quick reverse geolocator search, 
# # use a smaller resolution and create polygons for each location
# if len(weather_dropped_df.id_tuple.unique()) >2000 or force_polyrequest:
#     # set path to load/save the grid
#     path = '../data/01_raw/address_grid_'+try_id+'.pkl'
#     if os.path.exists(path) and not force_new_grid:
#         # load the grid if possible
#         with open(path,'rb') as f:
#             grid_df=pickle.load(f)
#     else:
#         # request a grid if none is available
#         # reference the loc dict to get necessary coordinates
#         #...
#         lon_min = 9.8767193
#         lon_max = 12.6539178
#         lat_min = 50.2043467
#         lat_max = 51.6492842
#         reso = 24
#         # request the address grid with polygons
#         grid_df = address.get_address_grid(lat_min,lat_max,lon_min,lon_max,reso)
#         # save the grid for quicker access
#         with open(path,'wb') as f:
#             pickle.dump(grid_df,f)

In [ ]:
# this cell contains outdated code, only for documentation purposes

# # load or request address data
# # check if address data for the current round exists
# path = '../data/01_raw/address_data_raw' +'_' + try_id +'.csv'
# if os.path.exists(path) and use_available:
#     # if available, read currently used address data
#     address_df = pd.read_csv(path)
#     # turn id_tuple col into tuples
#     tups = address_df['id_tuple']
#     tups = tups.apply(lambda x: make_tuple(x))
#     address_df['id_tuple'] = tups
# else:
#     # if address data isn't available locally, request it from the api
#     address_df = address.get_address_data(ids_allowed)
#     # delete ids from ids_all_df if not in address_df
#     ids_all_df = ids_all_df[ids_all_df['id_tuple'].isin(address_df['id_tuple'])]
#     # also save it to reduce number of requests
#     address_df.to_csv(path, index=False)

# # block locations not in germany
# block_df=idmanage.update_id_block_df(
#     address_df,
#     block_df,
#     'different from',
#     'de',
#     block_reason='not in germany',
#     insights=False,
#     block_col='country code'
# )
# # update ids_all_df and ids_allowed
# ids_all_df = idmanage.updateblocks_idmanage_df(block_df, ids_all_df)
# ids_allowed = ids_all_df[ids_all_df['blocked']==False]['id_tuple']

In [ ]:
# load or request elevation carpets
path = '../data/01_raw/carpet_pickled' +'_' + try_id +'.pkl'
# check if data is available
if os.path.exists(path) and use_available:
    # load carpet_list if possible
    with open(path,'rb') as f:
        carpet_list = pickle.load(f)
else:
    # else request and save carpet_list
    # list of carpet data for each allowed location: carpet_list
    carpet_list = []
    # request carpets
    for tup in ids_allowed:
        lat = tup[0]
        lon = tup[1]
        carpet_list.append(topo_raw.elevation_carpet(lat,lon))
    # save carpet_list in .pkl file
    with open(path,'wb') as f:
        pickle.dump(carpet_list,f)

In [ ]:
# turn all elevation carpets into elevation dataframes, save in df: elevation_df_list
# also get characteristics for each carpet, save in list: characteristics_list
elevation_df_list =[]
characteristics_list =[]
for carpet in carpet_list:
    elevation_df = topo_int.carpet2df(carpet)
    characteristics = topo_int.carpet_characterics(carpet)
    elevation_df_list.append(elevation_df)
    characteristics_list.append(characteristics)

In [ ]:
weather_reformatted_df['total_aod_550nm:idx value'].mean()

In [ ]:
# get the ns and ew gradients for each location (facing north), save in lists: ns_grad_list, ew_grad_list
# get standard deviation of elevation around each location, save in list: elevation_std_list
ns_grad_list = []
ew_grad_list = []
elevation_std_list = []
for i in range(len(elevation_df_list)):
    ns_grad_list.append(topo_int.ground_grad_ns(elevation_df_list[i],characteristics_list[i]['ns_stepsize']))
    ew_grad_list.append(topo_int.ground_grad_ew(elevation_df_list[i],characteristics_list[i]['ew_stepsize']))
    elevation_std_list.append(topo_int.elevation_std(elevation_df_list[i]))
# collect intermediate (preprocessed) carpet data in dataframe
carpet_int = {'ns grad':ns_grad_list,'ew grad':ew_grad_list,'elevation std':elevation_std_list,'id_tuple':ids_allowed}
carpet_data_int_df = pd.DataFrame(carpet_int)
carpet_data_int_df

In [ ]:
# update blocked ids based on the value of the ns_gradient
block_df = idmanage.update_id_block_df(
    carpet_data_int_df,
    block_df,
    'less than',
    -0.09,
    'steiler Hang Richtung Norden',
    block_col='ns grad'
)
# update ids_all_df and ids_allowed
ids_all_df = idmanage.updateblocks_idmanage_df(block_df, ids_all_df).reset_index(drop=True)
ids_allowed = ids_all_df[ids_all_df['blocked']==False]['id_tuple'].reset_index(drop=True)

In [ ]:
# get intermediate topo data that is not blocked
carpet_data_int_df=carpet_data_int_df[carpet_data_int_df['id_tuple'].isin(block_df['id_tuple'])==False].reset_index(drop=True)
# add cols for tilt and alignment, calculate the values
carpet_data_int_df['tilt'] = carpet_data_int_df.apply(lambda x: math.degrees(math.tan(x['ns grad'])), axis=1)
carpet_data_int_df['alignment'] = carpet_data_int_df.apply(lambda x: topo_int.grads_to_alignment(x['ns grad'],x['ew grad']), axis=1)
carpet_data_int_df

In [ ]:
# get or load raw elevation_path data looking east and west for each location, also get distance between points on path in meters
path = '../data/01_raw/ele_path_eastwest_pickled' +'_' + try_id +'.pkl'
if os.path.exists(path) and use_available:
    # load elevation path data if available locally
        with open(path,'rb') as f:
            ele_path_list_dict = pickle.load(f)
            # load path lists if available
            ele_path_east_list = ele_path_list_dict.get('east',math.nan)
            ele_path_west_list = ele_path_list_dict.get('west',math.nan)
            step_size_list = ele_path_list_dict.get('stepsizes',math.nan)
else:
    # request and save elevation path data if unavailable
    # create empty lists: ele_path_east_list, ele_path_west_list
    ele_path_east_list = []
    ele_path_west_list = []
    # empty list to save stepsizes between points on path in meter: step_size_list
    step_size_list = []
    for tup in ids_allowed:
        # request all elevation paths for allowed locations
        lat = tup[0]
        lon = tup[1]
        # get the path profile (only elevation values)
        ele_path_east = topo_raw.elevation_path(lat,lon,direction='east').get('profile')
        ele_path_west = topo_raw.elevation_path(lat,lon,direction='west').get('profile')
        # compute stepsize
        ew_stepsize = math.cos(lat)*111.325
        # all elevation path to list of elevation paths
        ele_path_east_list.append(ele_path_east)
        ele_path_west_list.append(ele_path_west)
        step_size_list.append(ew_stepsize)
    ele_path_list_dict = {'east':ele_path_east_list, 'west':ele_path_west_list,'stepsizes':step_size_list}
    # save pickled elevation path data
    with open(path,'wb') as f:
        pickle.dump(ele_path_list_dict,f)

In [ ]:
# get the highest available gradient from starting point to another point on the elevation path

# empty lists to store highest gradients
max_grad_east_list, max_grad_west_list = [], []
# iterate over all paths to find the highest gradient in each direction
# can be improved by including a variable list of path lists, eg by iterating over the keys of a path_list_dict
# not needed since there are only to directions of interest
for i in range(len(ele_path_east_list)):
    # get the paths, stepsize (distance between path points in m)
    path_east = ele_path_east_list[i]
    path_west = ele_path_west_list[i]
    stepsize = step_size_list[i]
    # get the max gradient in each path
    max_grad_east = topo_int.maxgrad_inpath(path_east,stepsize)
    max_grad_west = topo_int.maxgrad_inpath(path_west,stepsize)
    # append the max gradient to the corresponding list
    max_grad_east_list.append(max_grad_east)
    max_grad_west_list.append(max_grad_west)
# save the max gradient data in dataframe: max_grad_df
# use max_grad_dict for easier creation of the df
max_grad_dict = {'max grad west':max_grad_west_list, 'max grad east':max_grad_east_list}
max_grad_df = pd.DataFrame.from_dict(max_grad_dict).set_index(ids_allowed,drop=True)

In [ ]:
# turn the gradients into degrees
max_grad_df['max west degrees'] = max_grad_df.apply(lambda x: math.atan(x['max grad west'])*180/math.pi, axis=1)
max_grad_df['max east degrees'] = max_grad_df.apply(lambda x: math.atan(x['max grad east'])*180/math.pi, axis=1)

In [ ]:
# get the sunheights for different times at the locations
# trying to find data around sunrise and sunset times
# save results in dataframe
# set parameters for sunrise
starttime = dt.datetime(2000,1,1,hour=4,minute=45)
minute_delta = dt.timedelta(seconds=300)
n_times = 12
date_str = '2021-09-20 '

# create empty lists for sunrises and corresponding times
sunrise_list, idx_times = [], []
# iterate over requested times
for i in range(n_times):
    # iterate over location tuples, get sunrises for each
    for tup in max_grad_df.index:
        lat = tup[0]
        lon = tup[1]
        # generate datetime_str for the sh.sunrise function
        time_str = dt.datetime.strftime(starttime + i*minute_delta,'%H:%M')
        datetime_str = date_str + time_str
        # compute sunrise for given time and location
        sunrise = sh.sunheight(lat,lon,datetime_str)
        # save the sunrise and time string
        sunrise_list.append(sunrise)
    idx_times.append(time_str)
# reshape sunrise list, put into dataframe, use times for columns

# reshape the sunrise list so it's one col per location
array = np.array(sunrise_list).reshape(n_times,-1)
# turn the array into a dataframe, set fitting index and col names
sunrise_df = pd.DataFrame(array,index=idx_times)
sunrise_df.columns = max_grad_df.index



# set parameters for sunset
starttime = dt.datetime(2000,1,1,hour=16,minute=45)
minute_delta = dt.timedelta(seconds=300)
n_times = 12
date_str = '2021-09-20 '

# create empty lists for sunrises and corresponding times
sunset_list, idx_times = [], []
# iterate over requested times
for i in range(n_times):
    # iterate over location tuples, get sunrises for each
    for tup in max_grad_df.index:
        lat = tup[0]
        lon = tup[1]
        # generate datetime_str for the sh.sunrise function
        time_str = dt.datetime.strftime(starttime + i*minute_delta,'%H:%M')
        datetime_str = date_str + time_str
        # compute sunrise for given time and location
        sunset = sh.sunheight(lat,lon,datetime_str)
        # save the sunrise and time string
        sunset_list.append(sunset)
    idx_times.append(time_str)
# reshape sunrise list, put into dataframe, use times for columns

# reshape the sunrise list so it's one col per location
array = np.array(sunset_list).reshape(n_times,-1)
# turn the array into a dataframe, set fitting index and col names
sunset_df = pd.DataFrame(array,index=idx_times)
sunset_df.columns = max_grad_df.index
# sort sunset_df with ascending degree values
sunset_df.sort_values(by=sunset_df.columns[0],inplace=True)


In [ ]:
# empty lists to save effective and real sunrise and sunset
sr_eff_list, sr_real_list, ss_eff_list, ss_real_list = [],[], [], []

# iterate over locations in max_grad_df
# get real and effective sunrise
for tup in max_grad_df.index:
    
    # if there's shaded time after sunrise (if the degrees value is positive) compute both real and effective sunrise time
    if max_grad_df['max east degrees'][tup] >0:
        # get the sunrise times
        sr_eff_pos = bisect.bisect(sunrise_df[tup],max_grad_df['max east degrees'][tup])
        sr_real_pos = bisect.bisect(sunrise_df[tup],0)
        sr_eff = sunrise_df.index[sr_eff_pos]
        sr_real = sunrise_df.index[sr_real_pos]
        # save the sunrise times
        sr_eff_list.append(sr_eff)
        sr_real_list.append(sr_real)
    else:
        # if there's no difference between real and effective sunrise, save the real sunrise for both
        sr_real_pos = bisect.bisect(sunrise_df[tup],0)
        sr_real = sunrise_df.index[sr_real_pos]
        sr_real_list.append(sr_real)
        sr_eff_list.append(sr_real)
    # if there's shaded time just before sunset (if the degrees value is positive) compute both real and effective sunset time
    if max_grad_df['max west degrees'][tup] >0:
        # get the sunset times
        ss_eff_pos = bisect.bisect(sunset_df[tup],max_grad_df['max west degrees'][tup])
        ss_real_pos = bisect.bisect(sunset_df[tup],0)
        ss_eff = sunset_df.index[ss_eff_pos]
        ss_real = sunset_df.index[ss_real_pos]
        # save the sunrise times
        ss_eff_list.append(ss_eff)
        ss_real_list.append(ss_real)
    else:
        # if there's no difference between real and effective sunset, save the real sunset for both
        ss_real_pos = bisect.bisect(sunset_df[tup],0)
        ss_real = sunset_df.index[ss_real_pos]
        ss_real_list.append(ss_real)
        ss_eff_list.append(ss_real)


# save sunrise data in dataframe: sunrise_sunset_timeloss_df
sunrise_sunset_timeloss_df = pd.DataFrame()
sunrise_sunset_timeloss_df.index = max_grad_df.index
sunrise_sunset_timeloss_df['real sunrise'] = sr_real_list
sunrise_sunset_timeloss_df['effective sunrise'] = sr_eff_list
sunrise_sunset_timeloss_df['real sunset'] = ss_real_list
sunrise_sunset_timeloss_df['effective sunset'] = ss_eff_list


In [ ]:
sunrise_sunset_timeloss_df

In [ ]:
to_datetime = lambda x: dt.datetime.strptime(x,'%H:%M')
# compute time lost by shade just after sunrise and just before sunset
timeloss_help_df = sunrise_sunset_timeloss_df.applymap(to_datetime)
# calculate day lengths
timeloss_help_df['real day length'] = timeloss_help_df.apply(lambda x: x['real sunset']-x['real sunrise'], axis=1)
timeloss_help_df['effective day length'] = timeloss_help_df.apply(lambda x: x['effective sunset']-x['effective sunrise'], axis=1)
# calculate absolute and relative losses of sun minutes by shade thrown by nearby elevations
timeloss_help_df['abs time loss by elevationshade'] = timeloss_help_df.apply(lambda x: x[4]-x[5], axis=1)
timeloss_help_df['rel time loss by elevationshade'] = timeloss_help_df.apply(lambda x: x[6]/x[4], axis=1)
# add the loss of sun minutes to sunrise_sunset_timeloss_df
sunrise_sunset_timeloss_df[['abs time loss by elevationshade','rel time loss by elevationshade']] = timeloss_help_df[['abs time loss by elevationshade','rel time loss by elevationshade']]

In [ ]:
# get reduced df with id and loss pct: elevationshade_red_df
elevationshade_red_df= sunrise_sunset_timeloss_df[['rel time loss by elevationshade']]
elevationshade_red_df.columns = ['loss by elevationshade']
elevationshade_red_df

In [ ]:
# block ids based on max gradients if necessary, update id management dfs, save updated id_all_df

In [ ]:
# get processed topo data with expected losses
# make sure only allowed locations are processed
carpet_data_int_df=carpet_data_int_df[carpet_data_int_df['id_tuple'].isin(block_df['id_tuple'])==False].reset_index(drop=True)
# get expected losses by tilt and alignment
# load the efficiency dataframe
path = '../references/Efficiency of solar panels by tilt and alignment in germany.xlsx'
efficiency_df = pd.read_excel(path, index_col=0)
# calculate the losses and save in topo_loss_tiltalign_df
topo_loss_tiltalign_df = pd.DataFrame()
topo_loss_tiltalign_df['id_tuple'] = carpet_data_int_df['id_tuple']
topo_loss_tiltalign_df['loss by tilt, alignment'] = carpet_data_int_df.apply(lambda x: topo_pro.tiltalign_loss_lowres(x.tilt,x.alignment,efficiency_df),axis=1)
topo_loss_tiltalign_df

In [ ]:
# create geodataframe with all allowed locations, all losses and geometries for each position

# instantiate loss_df with all losses
loss_df = topo_loss_tiltalign_df
if 'id_tuple' in loss_df.columns:
    loss_df.set_index('id_tuple',drop=True,inplace=True)
# join loss by elevationshade
loss_df = loss_df.join(elevationshade_red_df)
# calculate total loss percentage
loss_df['total losses'] = loss_df.apply(lambda x: 1-(1-x[0])*(1-x[1]),axis=1)
# turn loss_df into geodataframe with polygons
# get polygons first
loss_gdf = creategeo.create_polygons(loss_df)
# change type to gdf
crs = 'epsg:4326'
loss_gdf = gpd.GeoDataFrame(loss_gdf, crs=crs, geometry=loss_gdf.geometry)
loss_gdf